# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge

# Read Data

In [2]:
data = pd.read_csv('../01-Data/2004-2019.tsv', sep='\t', parse_dates=['DATA INICIAL', 'DATA FINAL'])

In [3]:
data.head()

,Unnamed: 0,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO,MÊS,ANO
0,0,2004-05-09,2004-05-15,CENTRO OESTE,DISTRITO FEDERAL,ETANOL HIDRATADO,127,R$/l,1.288,0.016,1.19,1.350,0.463,0.012,0.825,0.11,0.4201,0.9666,0.133,5,2004
1,1,2004-05-09,2004-05-15,CENTRO OESTE,GOIAS,ETANOL HIDRATADO,387,R$/l,1.162,0.114,0.89,1.449,0.399,0.098,0.763,0.088,0.5013,1.05,0.115,5,2004
2,2,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO,ETANOL HIDRATADO,192,R$/l,1.389,0.097,1.18,1.760,0.419,0.070,0.97,0.095,0.5614,1.161,0.098,5,2004
3,3,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO DO SUL,ETANOL HIDRATADO,162,R$/l,1.262,0.070,1.09,1.509,0.432,0.055,0.83,0.119,0.5991,1.22242,0.143,5,2004
4,4,2004-05-09,2004-05-15,NORDESTE,ALAGOAS,ETANOL HIDRATADO,103,R$/l,1.181,0.078,1.05,1.400,0.24,0.066,0.941,0.077,0.7441,1.0317,0.082,5,2004


# Translate column names to English

In [4]:
data.columns

Index(['Unnamed: 0', 'DATA INICIAL', 'DATA FINAL', 'REGIÃO', 'ESTADO',
       'PRODUTO', 'NÚMERO DE POSTOS PESQUISADOS', 'UNIDADE DE MEDIDA',
       'PREÇO MÉDIO REVENDA', 'DESVIO PADRÃO REVENDA', 'PREÇO MÍNIMO REVENDA',
       'PREÇO MÁXIMO REVENDA', 'MARGEM MÉDIA REVENDA',
       'COEF DE VARIAÇÃO REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO',
       'DESVIO PADRÃO DISTRIBUIÇÃO', 'PREÇO MÍNIMO DISTRIBUIÇÃO',
       'PREÇO MÁXIMO DISTRIBUIÇÃO', 'COEF DE VARIAÇÃO DISTRIBUIÇÃO', 'MÊS',
       'ANO'],
      dtype='object')

In [5]:
data.columns = [
                    "Unnamed:_0", 
                    "Analysis_Date", #DATA INICIAL
                    "Last_Day_of_Analyses_of_Week", #DATA FINAL
                    "Macroregion", #REGIÃO
                    "State", #ESTADO
                    "Product", #PRODUTO
                    "No_of_Gas_Stations_Analyzed", #NÚMERO DE POSTOS PESQUISADOS
                    "Measurement_Unit", #UNIDADE DE MEDIDA
                    "Mean_Price", #PREÇO MÉDIO REVENDA
                    "Std_Dev", #MARGEM MÉDIA REVENDA
                    "Min_Price", #DESVIO PADRÃO REVENDA
                    "Max_Price", #PREÇO MÁXIMO REVENDA
                    "Mean_Price_Margin", #MARGEM MÉDIA REVENDA
                    "Coefficient_of_variation", #COEF DE VARIAÇÃO REVENDA
                    "Mean_Dist_Price", #PREÇO MÉDIO DISTRIBUIÇÃO
                    "Distribution_Std_Dev", #DESVIO PADRÃO DISTRIBUIÇÃO
                    "Distribution_Min_Price", #PREÇO MÍNIMO DISTRIBUIÇÃO
                    "Distribution_Max_Price", #PREÇO MÁXIMO DISTRIBUIÇÃO
                    "Distribution_Coefficient_of_Variation", #COEF DE VARIAÇÃO DISTRIBUIÇÃO
                    "Month", #MÊS
                    "Year" #ANO
]

In [6]:
data.columns

Index(['Unnamed:_0', 'Analysis_Date', 'Last_Day_of_Analyses_of_Week',
       'Macroregion', 'State', 'Product', 'No_of_Gas_Stations_Analyzed',
       'Measurement_Unit', 'Mean_Price', 'Std_Dev', 'Min_Price', 'Max_Price',
       'Mean_Price_Margin', 'Coefficient_of_variation', 'Mean_Dist_Price',
       'Distribution_Std_Dev', 'Distribution_Min_Price',
       'Distribution_Max_Price', 'Distribution_Coefficient_of_Variation',
       'Month', 'Year'],
      dtype='object')

# Train and Validation Split (Simple Holdout)

In [7]:
data_train = data[data['Last_Day_of_Analyses_of_Week'] < '2011-01-01']
data_valid = data[data['Last_Day_of_Analyses_of_Week'] >= '2011-01-01']

data_train.shape, data_valid.shape

((42514, 21), (64309, 21))

# New DataFrame for Train and Validation (Index: original Data)

In [8]:
df_train = pd.DataFrame(index=data_train.index)
df_valid  = pd.DataFrame(index=data_valid.index)

# Target

## First Difference of Average Resale Price

In [9]:
df_train['diff_Mean_Price'] = data_train.groupby(['Product', 'State'])['Mean_Price'].apply(lambda row: row.diff().shift(-1))
df_valid['diff_Mean_Price'] = data_valid.groupby(['Product', 'State'])['Mean_Price'].apply(lambda row: row.diff().shift(-1))

# Features

## Current Mean Price

In [10]:
df_train['Current_Mean_Price'] = data_train['Mean_Price']
df_valid['Current_Mean_Price'] = data_valid['Mean_Price']

## Seasonality

In [11]:
df_train['month'] = data_train['Last_Day_of_Analyses_of_Week'].dt.month
df_train['day'] = data_train['Last_Day_of_Analyses_of_Week'].dt.day
df_train['weekday'] = data_train['Last_Day_of_Analyses_of_Week'].dt.weekday
df_train['dayofyear'] = data_train['Last_Day_of_Analyses_of_Week'].dt.dayofyear

df_valid['month'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.month
df_valid['day'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.day
df_valid['weekday'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.weekday
df_valid['dayofyear'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.dayofyear

## Movel Average

In [12]:
roll = data_train.groupby(['Product', 'State'])['Mean_Price'].rolling(4).mean()
roll.reset_index(level=[0,1], drop=True).head()

13         NaN
40         NaN
67         NaN
94     1.80525
121    1.80550
Name: Mean_Price, dtype: float64

In [13]:
df_train['Movel_Average_Mean_Price_4_weeks'] = data_train.groupby(['Product', 'State'])['Mean_Price'].rolling(4).mean().reset_index(level=[0,1], drop=True)
df_valid['Movel_Average_Mean_Price_4_weeks'] = data_valid.groupby(['Product', 'State'])['Mean_Price'].rolling(4).mean().reset_index(level=[0,1], drop=True)

In [14]:
df_train.isnull().sum()

diff_Mean_Price                     134
Current_Mean_Price                    0
month                                 0
day                                   0
weekday                               0
dayofyear                             0
Movel_Average_Mean_Price_4_weeks    401
dtype: int64

## One Hot Encoding >> Product

In [15]:
## Unique values in the Product Column 
data_train['Product'].unique()

array(['ETANOL HIDRATADO', 'GASOLINA COMUM', 'GLP', 'GNV', 'ÓLEO DIESEL'],
      dtype=object)

In [16]:
## Unique values in the Product Column
data_valid['Product'].unique()

array(['ETANOL HIDRATADO', 'GASOLINA COMUM', 'GLP', 'GNV', 'ÓLEO DIESEL',
       'ÓLEO DIESEL S10'], dtype=object)

#### Selecting only the values that exist in the dataset training to follow with the one hot encoding

In [19]:
data_valid = data_valid[data_valid['Product'].isin(data_train['Product'].unique())]

data_train.shape, data_valid.shape

((42514, 21), (55196, 21))

In [20]:
df_train = df_train.join(pd.get_dummies(data_train['Product'], drop_first=True))
df_valid = df_valid.join(pd.get_dummies(data_valid['Product'], drop_first=True))

## Drop Missing Values

In [22]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

## X, y Train and Validation Split

In [23]:
Xtr, ytr = df_train.drop(['diff_Mean_Price'], axis=1), df_train['diff_Mean_Price']
Xval, yval = df_valid.drop(['diff_Mean_Price'], axis=1), df_valid['diff_Mean_Price']

# Model

## RandomForestRegressor

In [24]:
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=500)
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [25]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

2.224409413527068

## LGBMRegressor

In [26]:
mdl = LGBMRegressor(num_leaves=2, min_data_in_leaf=250, n_jobs=-1, random_state=0, n_estimators=500)
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

[LightGBM] [Warning] min_data_in_leaf is set=250, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=250


#### Metric

In [27]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2895354652244877

## LinearRegression

In [28]:
mdl = LinearRegression()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [29]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2710134332649696

## Lasso

In [30]:
mdl = Lasso()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [31]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2879556493823672

## Ridge

In [32]:
mdl = Ridge()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [33]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2709157369709527

## BayesianRidge

In [34]:
mdl = BayesianRidge()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [35]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2708908878890794